# DataLoader Setup

In [1]:
import os
import torch
import torchvision
from pytorch_lightning import seed_everything
from pl_bolts.datamodules import CIFAR10DataModule, TinyCIFAR10DataModule
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization

PATH_DATASETS = os.environ.get('PATH_DATASETS', '../datasets')
AVAIL_GPUS = min(1, torch.cuda.device_count())
BATCH_SIZE = 512 if AVAIL_GPUS else 64
NUM_WORKERS = int(os.cpu_count()/2)
TEST_ONLY = False

test_transforms = [
        torchvision.transforms.ToTensor(),
        cifar10_normalization(),        
        torchvision.transforms.Lambda(lambda x: x.clamp(min = -1, max = 1)),
        torchvision.transforms.Lambda(lambda x: x * 127),   
        torchvision.transforms.Lambda(lambda x: x.floor())  
]

train_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.RandomCrop(32, padding=4),
        torchvision.transforms.RandomHorizontalFlip(),
    ] 
    + test_transforms
)

test_transforms = torchvision.transforms.Compose(test_transforms)

cifar10_dm = CIFAR10DataModule(
    data_dir = PATH_DATASETS,
    batch_size = BATCH_SIZE,
    num_workers = NUM_WORKERS,
    train_transforms = train_transforms,
    test_transforms = test_transforms,
    val_transforms = test_transforms,
)

seed_everything(seed=1234, workers=True)

/home/neungyun/anaconda3/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:73: LightningDeprecationWarning: DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
/home/neungyun/anaconda3/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:77: LightningDeprecationWarning: DataModule property `val_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
/home/neungyun/anaconda3/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:81: LightningDeprecationWarning: DataModule property `test_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
Global seed set to 1234


1234

# model 

In [2]:
from model import Model

teacher_model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_resnet44", pretrained=True)

# model_structure = [32] + [64] + [128] * 2 + [256] * 2 + [512] * 6 + [1024] * 2
 
model_structure =  [256] * 2 + [512] * 6 + [1024] * 2
 

react_model = Model(
    structure=model_structure,
    adam_init_lr=1e-2, 
    adam_weight_decay=0,
    adam_betas=(0.9, 0.999),
    lr_reduce_factor=0.1,
    lr_patience=50,
    limit_conv_weight=True,
    limit_bn_weight=True,
    teacher_model=None
    )

print(react_model.hparams)
print(react_model)

Using cache found in /home/neungyun/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master


"adam_betas":        (0.9, 0.999)
"adam_init_lr":      0.01
"adam_weight_decay": 0
"limit_bn_weight":   True
"limit_conv_weight": True
"lr_patience":       50
"lr_reduce_factor":  0.1
"structure":         [256, 256, 512, 512, 512, 512, 512, 512, 1024, 1024]
"teacher_model":     None
Model(
  (blocks): ModuleList(
    (0): firstconv3x3(
      (conv1): Conv2d(3, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): UpperBlock(
      (conv): GeneralConv2d(256, 256, kernel_size=3, stride=1, padding=1, conv=scaled_sign)
      (rsign): RSign(in_channels=256)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (rprelu): RPReLU(in_channels=256)
    )
    (2): LowerBlock(
      (rsign): RSign(in_channels=256)
      (conv): GeneralConv2d(256, 256, kernel_size=1, stride=1, padding=0, conv=scaled_sign)
      (bn): BatchNorm2d(256,

In [3]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

checkpoint_callback = ModelCheckpoint(filename='{epoch}-{val_loss:.4f}-{val_acc:.4f}', monitor='val_acc', mode='max')
trainer = Trainer(
    max_epochs=-1,
    gpus=AVAIL_GPUS,
    logger=TensorBoardLogger('lightning_logs/', name='new_react_net_test_shape', log_graph=True),
    # logger=TensorBoardLogger('lightning_logs/', name='Real', log_graph=True),
    callbacks=[LearningRateMonitor(logging_interval='step'), 
               EarlyStopping(monitor='val_acc', mode='max', patience=100),
               checkpoint_callback],
    deterministic=True,
#    gradient_clip_val = 0.5
)

trainer.fit(react_model, datamodule=cifar10_dm)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Files already downloaded and verified
Files already downloaded and verified


/home/neungyun/anaconda3/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:114: LightningDeprecationWarning: DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
/home/neungyun/anaconda3/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:133: LightningDeprecationWarning: DataModule property `val_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/neungyun/anaconda3/lib/python3.9/site-packages/torch/_jit_internal.py:668: LightningDeprecationWarning: The `LightningModule.loaded_optimizer_states_dict` property is deprecated in v1.4 and will be removed in v1.6.
  if hasattr(mod, name):

  | Name   | Type              | Params | In sizes          | Out sizes        
-------------------------------------------------------------------------------------
0 | blocks | ModuleList        | 27.9 M | ?                 | ?    

Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 1234


...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [4]:
react_model.state_dict()

OrderedDict([('blocks.0.conv1.weight',
              tensor([[[[-9.8517e-01, -2.2227e-01,  3.6564e-01],
                        [-1.1794e+00, -1.2505e+00, -9.8036e-02],
                        [ 2.1342e-01, -1.1216e+00, -6.7963e-01]],
              
                       [[-6.4151e-01, -1.2253e+00, -9.2975e-01],
                        [ 9.2550e-02, -9.0942e-01, -7.4550e-01],
                        [ 3.7309e-01, -1.0714e+00, -9.2015e-01]],
              
                       [[-3.5030e-01,  5.2908e-01,  6.1031e-01],
                        [ 2.6113e-02,  4.9892e-01,  6.9306e-01],
                        [ 7.1329e-01, -9.6616e-01, -1.1444e+00]]],
              
              
                      [[[-6.8818e-01, -1.0897e+00, -4.3929e-01],
                        [ 1.3890e+00, -1.0257e+00, -5.4040e-01],
                        [ 1.1887e+00, -2.3569e-02,  1.1210e+00]],
              
                       [[ 7.2825e-02, -6.5297e-01,  1.7663e-01],
                        [ 6.1024e-01

In [5]:
trainer.test(react_model,cifar10_dm)


/home/neungyun/anaconda3/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:152: LightningDeprecationWarning: DataModule property `test_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.8736000061035156, 'test_loss': 1.151261568069458}
--------------------------------------------------------------------------------


[{'test_loss': 1.151261568069458, 'test_acc': 0.8736000061035156}]

In [6]:
print(checkpoint_callback.best_model_path)
print(checkpoint_callback.best_model_score)

lightning_logs/new_react_net_test_shape/version_6/checkpoints/epoch=386-val_loss=1.0155-val_acc=0.8846.ckpt
tensor(0.8846, device='cuda:0')
